# Setup

Ensure all dependencies are installed.


In [1]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Check if we have GPU support, and if not, warn the user.


In [2]:
import torch
import warnings

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    warnings.warn("CUDA is not available. Running on CPU.")

CUDA is available!


Load our dataset...


In [1]:
import tempfile
import os
import zipfile
import shutil

import utils.file

from data_sources import DataSource, ModelNet40
from pointnet.dataset import ModelNetDataset

data_source: DataSource = ModelNet40()
data_size = utils.file.get_download_size(data_source.url)

temp_dir = tempfile.gettempdir()
zip_path = os.path.join(temp_dir, data_source.name + ".zip")
dataset_path = os.path.join(temp_dir, data_source.name)

if os.path.exists(zip_path) and utils.file.checksum(zip_path) == data_source.checksum:
    print("Zip already downloaded.")
else:
    utils.file.download_file(data_source.url, zip_path)
    print("Zip downloaded.")

if os.path.exists(dataset_path):
    print("Conflicting data folder found. Deleting existing folder...")
    shutil.rmtree(dataset_path)
    print("Folder deleted.")

print("Extracting data...")
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(dataset_path)
    print("Data extracted.")

npoints = 2500

train_dataset = ModelNetDataset(root=dataset_path, npoints=npoints, split="train")

test_dataset = ModelNetDataset(
    root=dataset_path,
    split="test",
    npoints=npoints,
)

print(
    f"Training data: {len(train_dataset)} samples with {len(train_dataset.classes)} classes"
)
print(
    f"Test data size: {len(test_dataset)} samples with {len(test_dataset.classes)} classes"
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Zip already downloaded.
Conflicting data folder found. Deleting existing folder...
Folder deleted.
Extracting data...
Data extracted.
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/chair/train/chair_0856.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/chair/train/chair_0092.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/chair/train/chair_0452.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/chair/train/chair_0451.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/table/train/table_0335.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/table/train/table_0378.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/table/train/table_0241.off
Fixing OFF file format for /tmp/ModelNet40/ModelNet40/table/train/table_0261.off
Fixing OFF file format for /tmp/ModelNet40/ModelN

# Training

Train a new classifier or load a new pre-trained one:

In [4]:
from pointnet.train_classification import train_classification

classifier_path = "./models/classifier_mn40.pth"

if classifier_path:
    
else:
    train_classification(train_dataset=train_dataset, test_dataset=test_dataset, epochs=25)

Random Seed:  8403
[0: 0/307] train loss: 3.941219 accuracy: 0.000000
[0: 0/307] test loss: 3.682500 accuracy: 0.125000
[0: 1/307] train loss: 3.673783 accuracy: 0.125000
[0: 2/307] train loss: 3.714046 accuracy: 0.125000
[0: 3/307] train loss: 3.607891 accuracy: 0.093750
[0: 4/307] train loss: 3.392532 accuracy: 0.156250
[0: 5/307] train loss: 3.522344 accuracy: 0.093750
[0: 6/307] train loss: 3.479636 accuracy: 0.125000
[0: 7/307] train loss: 3.168527 accuracy: 0.281250
[0: 8/307] train loss: 2.916125 accuracy: 0.468750
[0: 9/307] train loss: 2.902792 accuracy: 0.312500
[0: 10/307] train loss: 3.192945 accuracy: 0.375000
[0: 10/307] test loss: 3.810632 accuracy: 0.031250
[0: 11/307] train loss: 2.969747 accuracy: 0.281250
[0: 12/307] train loss: 2.960296 accuracy: 0.343750
[0: 13/307] train loss: 2.868739 accuracy: 0.281250
[0: 14/307] train loss: 2.890066 accuracy: 0.343750
[0: 15/307] train loss: 2.761026 accuracy: 0.312500
[0: 16/307] train loss: 2.568478 accuracy: 0.375000
[0: 17

78it [00:19,  4.04it/s]

final accuracy 0.8500810372771475


Train the segmenter:


In [3]:
from pointnet.train_segmentation import train_segmentation

train_segmentation(train_dataset=train_dataset, test_dataset=test_dataset, epochs=25, class_choice=0)

Random Seed:  9583


ValueError: Expected input batch_size (80000) to match target batch_size (32).